In [17]:
begin
	import Pkg
	Pkg.activate("..")
	Pkg.instantiate()
	
    push!(LOAD_PATH, "$(@__DIR__)/../src")
	using NoisyReach

    using ControlSystemsBase
    using LinearAlgebra
    using ReachabilityAnalysis
end

  Activating project at `~/Downloads/NoisyReach.jl`


In [18]:
const Ts = 0.01 # Sampling period

0.01

In [19]:
const Dc = 0.005 # sensor-to-actuator delay

0.005

In [20]:
# Example from page 33 of lecture 17 (Ts = 0.01, Dc = 0.005)
# Expected:
# ϕ = [1 0.001; -0.001 0.999]
# Γ₁ = [0; 0.0005]
# Γ₀ = [0; 0.0005]
#
# ϕ_aug = [1 0.001 0; -0.001 0.999 0.0005; 0 0 0]
# Γ_aug = [0 0.0005 1]
# C_aug = [1 0 0]
sys = let
    A = [0 1; -1 -1]
    B = [0; 1]
    C = [1 0]
    D = 0
    ss(A, B, C, D)
end

StateSpace{Continuous, Int64}
A = 
  0   1
 -1  -1
B = 
 0
 1
C = 
 1  0
D = 
 0

Continuous-time state-space model

In [21]:
sys_aug_ = let
    ϕ = ℯ^(Ts * sys.A)
    Γ₁ = Dc * sys.B
    Γ₀ = (Ts - Dc) * sys.B
    ϕ_aug = [ϕ Γ₁; 0 0 0]
    Γ_aug = [Γ₀; I]
    C_aug = [sys.C 0]
    ss(ϕ_aug, Γ_aug, C_aug, sys.D, Ts)
end

StateSpace{Discrete{Float64}, Float64}
A = 
  0.9999501666658346    0.009950000415833335  0.0
 -0.009950000415833335  0.9900001662500014    0.005
  0.0                   0.0                   0.0
B = 
 0.0
 0.005
 1.0
C = 
 1.0  0.0  0.0
D = 
 0.0

Sample Time: 0.01 (seconds)
Discrete-time state-space model

In [22]:
#sys_aug = c2d(sys, Ts) * delay(Dc, Ts)
sys_aug = c2d(sys, Ts) * thiran(Dc, Ts)

StateSpace{Discrete{Float64}, Float64}
A = 
  0.9999501666658346    0.009950000415833335   4.429629703580247e-5
 -0.009950000415833335  0.9900001662500014     0.008844444814074077
  0.0                   0.0                   -0.3333333333333333
B = 
 1.661111138842593e-5
 0.0033166668052777787
 1.0
C = 
 1.0  0.0  0.0
D = 
 0.0

Sample Time: 0.01 (seconds)
Discrete-time state-space model

In [23]:
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)

1×3 Matrix{Float64}:
 0.239321  0.418528  -0.167879